In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:22pt;}
div.output {font-size:22pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:22pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:22pt;padding:5px;}
table.dataframe{font-size:22px;}
</style>
"""))

In [2]:
# 패키지 import
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# 시각화의 선명도를 높임
%config InlineBackend.figure_format = "retina"

# 한글설정
sns.set(style='white', rc={'figure.figsize':(12,3)})
plt.rc('font', family='Malgun Gothic') # 윈도우즈
plt.rc('axes', unicode_minus=False) # 축의 - 깨짐 방지

In [7]:
path = r'C:/폐업율예측/data/6110000_서울특별시_07_24_04_P_일반음식점.csv'
df = pd.read_csv(path, encoding='cp949')

C:\Users\Admin\AppData\Local\Temp\ipykernel_12060\197698916.py:2: DtypeWarning: Columns (11,39,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, encoding='cp949')


In [8]:
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 527872 entries, 0 to 527871
Data columns (total 48 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   번호               527872 non-null  int64  
 1   개방서비스명           527872 non-null  object 
 2   개방서비스아이디         527872 non-null  object 
 3   개방자치단체코드         527872 non-null  int64  
 4   관리번호             527872 non-null  object 
 5   인허가일자            527872 non-null  object 
 6   인허가취소일자          0 non-null       float64
 7   영업상태구분코드         527872 non-null  int64  
 8   영업상태명            527872 non-null  object 
 9   상세영업상태코드         527872 non-null  int64  
 10  상세영업상태명          527872 non-null  object 
 11  폐업일자             406206 non-null  object 
 12  휴업시작일자           0 non-null       float64
 13  휴업종료일자           0 non-null       float64
 14  재개업일자            0 non-null       float64
 15  소재지전화            313736 non-null  object 
 16  소재지면적            493642 non-null  floa

(527872, 48)

In [9]:
df['소재지전체주소'].isna().sum()

228

In [10]:
df['도로명전체주소'].isna().sum()

249945

In [11]:
# '소재지전체주소'가 결측치인 행(miss_so)의 '소재지전체주소' 컬럼에
# 같은 행의 '도로명전체주소' 값을 대입하여 결측치를 채웁니다.
miss_so = df['소재지전체주소'].isnull()
df.loc[miss_so, '소재지전체주소'] = df.loc[miss_so, '도로명전체주소']

In [12]:
# 보완 작업이 반영된 후, 이전 결측치 인덱스를 다시 확인
miss_so = df['소재지전체주소'].isnull()
df.loc[miss_so, ['소재지전체주소', '도로명전체주소']]

,소재지전체주소,도로명전체주소


In [13]:
# '소재지전체주소' 컬럼의 결측치 개수 확인
df['소재지전체주소'].isna().sum()

0

In [14]:
# '공백이 아닌 글자들이 나오다가 구로 끝나는 단어'를 추출
df['구'] = df['소재지전체주소'].str.extract(r'(\S+구)')

In [16]:
save_cols = [
    '인허가일자',
    '영업상태명',
    '상세영업상태코드',
    '상세영업상태명',
    '폐업일자',
    '구',
    '소재지전체주소',
    '도로명전체주소',
    '사업장명',
    '좌표정보x(epsg5174)',
    '좌표정보y(epsg5174)',
    '업태구분명'
]

df[save_cols].to_csv(
    r"C:\폐업율예측\data\서울특별시_일반음식점 정리_전처리.csv",
    index=False,
    encoding="cp949"
)

In [18]:
df = pd.read_csv(r'C:/폐업율예측/data/서울특별시_일반음식점 정리_전처리.csv', encoding='cp949')
df.head(1)

C:\Users\Admin\AppData\Local\Temp\ipykernel_12060\1330317500.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:/폐업율예측/data/서울특별시_일반음식점 정리_전처리.csv', encoding='cp949')


,인허가일자,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,구,소재지전체주소,도로명전체주소,사업장명,좌표정보x(epsg5174),좌표정보y(epsg5174),업태구분명
0,2024-05-30,영업/정상,1,영업,NaN,강북구,서울특별시 강북구 수유동 229-46,"서울특별시 강북구 도봉로87길 11, 지하1층 (수유동)",미라클,202125.400308,459553.074675,기타


In [19]:
df['구'].value_counts() / len(df)

구
강남구     0.095260
송파구     0.057961
마포구     0.055324
영등포구    0.052282
서초구     0.051465
강서구     0.044276
강동구     0.043660
관악구     0.043344
중구      0.041862
동대문구    0.041000
종로구     0.038413
구로구     0.036810
성북구     0.034154
서대문구    0.034050
광진구     0.034008
양천구     0.033468
중랑구     0.032902
은평구     0.032845
강북구     0.032165
용산구     0.030727
노원구     0.030464
성동구     0.028164
동작구     0.026974
금천구     0.025951
도봉구     0.022445
일산서구    0.000004
원미구     0.000002
영통구     0.000002
수정구     0.000002
일산동구    0.000002
분당구     0.000002
수영구     0.000002
단원구     0.000002
Name: count, dtype: float64

In [20]:
df['업태구분명'].value_counts()

업태구분명
한식                 219832
분식                  80759
경양식                 51542
기타                  50366
호프/통닭               37435
일식                  21232
중국식                 16977
정종/대포집/소주방          12958
통닭(치킨)               9427
까페                   7834
패스트푸드                3887
외국음식전문점(인도,태국등)      3527
뷔페식                  2935
식육(숯불구이)             2072
김밥(도시락)              1718
횟집                   1634
전통찻집                  823
감성주점                  599
탕류(보신용)               432
출장조리                  422
패밀리레스트랑               385
라이브카페                 368
냉면집                   346
복어취급                  120
키즈카페                  108
이동조리                  101
커피숍                     4
기타 휴게음식점                3
일반조리판매                  2
룸살롱                     1
식품등 수입판매업               1
간이주점                    1
다방                      1
제과점영업                   1
식품소분업                   1
Name: count, dtype: int64

In [21]:
df['상세영업상태명'].value_counts()

상세영업상태명
폐업    406210
영업    121662
Name: count, dtype: int64

In [22]:
# 업태 빈도
type_counts = df['업태구분명'].value_counts()

# 10개 이하 업태 목록
rare_types = type_counts[type_counts <= 10].index

rare_types

Index(['커피숍', '기타 휴게음식점', '일반조리판매', '룸살롱', '식품등 수입판매업', '간이주점', '다방', '제과점영업',
       '식품소분업'],
      dtype='object', name='업태구분명')

In [23]:
# 표본 10개 이하 업태 → 기타로 통합
df['업태_정리'] = df['업태구분명'].replace(rare_types, '기타')

In [ ]:
# ===============================
#  업태 그룹화 함수
# ===============================
def map_category(x):
    if pd.isna(x):
        return '기타'

    if x in ['한식', '탕류(보신용)', '냉면집', '식육(숯불구이)']:
        return '한식'

    if x in ['분식', '김밥(도시락)', '패스트푸드']:
        return '분식/간편식'

    if x in ['경양식', '패밀리레스트랑', '외국음식전문점(인도,태국등)', '뷔페식']:
        return '양식/외식'

    if x in ['중국식', '일식']:
        return '중·일식'

    if x in ['호프/통닭', '정종/대포집/소주방', '통닭(치킨)', '감성주점', '간이주점', '룸살롱']:
        return '주점/치킨'

    if x in ['까페', '커피숍', '전통찻집', '라이브카페', '다방', '키즈카페']:
        return '카페'
    
    if x in ['식품등 수입판매업']
        return '식품등 수입판매업'
    
    return '기타'

In [23]:
# 업태 그룹화
df['업태'] = map_category(df['업태구분명'])

NameError: name 'map_category' is not defined

In [24]:
#===============================
#  지도 시각화용 라이브러리 아나콘다에 설치.
#!pip install geopandas
# ===============================

import geopandas as gpd
gpd.__version__

'1.1.1'

In [45]:
data = {
    '상호명': [f'가게_{i}' for i in range(1, 101)],
    '구': np.random.choice(['강남구', '강동구', '강북구', '강서구', '관악구'], 100),
    '폐업_3년이내': np.random.randint(0, 2, 100)  # 0(유지) 또는 1(폐업)
}
df = pd.DataFrame(data)

# 2. 질문하신 코드로 gu_rate_map 생성
gu_rate_map = (
    df.groupby('구')['폐업_3년이내']
      .mean()
      .mul(100)
      .round(1)
      .reset_index()
)

# 3. 결과 확인
print(gu_rate_map.sort_values('폐업_3년이내', ascending=False))

     구  폐업_3년이내
4  관악구     60.0
0  강남구     57.1
1  강동구     50.0
3  강서구     47.8
2  강북구     22.7


In [46]:
seoul_map = gpd.read_file(
    "https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json"
)
seoul_map.head()

,code,name,name_eng,base_year,geometry
0,11250,강동구,Gangdong-gu,2013,"POLYGON ((127.1152 37.55753, 127.1188 37.55722..."
1,11240,송파구,Songpa-gu,2013,"POLYGON ((127.06907 37.52228, 127.07496 37.520..."
2,11230,강남구,Gangnam-gu,2013,"POLYGON ((127.05867 37.5263, 127.06907 37.5222..."
3,11220,서초구,Seocho-gu,2013,"POLYGON ((127.01397 37.52504, 127.01918 37.520..."
4,11210,관악구,Gwanak-gu,2013,"POLYGON ((126.98368 37.47386, 126.98464 37.469..."


In [47]:
# 지도 쪽 구 이름 컬럼 통일
seoul_map['구'] = seoul_map['name']

# merge
map_df = seoul_map.merge(
    gu_rate_map,
    on='구',
    how='left'
)

map_df

,code,name,name_eng,base_year,geometry,구,폐업_3년이내
0,11250,강동구,Gangdong-gu,2013,"POLYGON ((127.1152 37.55753, 127.1188 37.55722...",강동구,50.0
1,11240,송파구,Songpa-gu,2013,"POLYGON ((127.06907 37.52228, 127.07496 37.520...",송파구,NaN
2,11230,강남구,Gangnam-gu,2013,"POLYGON ((127.05867 37.5263, 127.06907 37.5222...",강남구,57.1
3,11220,서초구,Seocho-gu,2013,"POLYGON ((127.01397 37.52504, 127.01918 37.520...",서초구,NaN
4,11210,관악구,Gwanak-gu,2013,"POLYGON ((126.98368 37.47386, 126.98464 37.469...",관악구,60.0
5,11200,동작구,Dongjak-gu,2013,"POLYGON ((126.98224 37.50931, 126.98242 37.501...",동작구,NaN
6,11190,영등포구,Yeongdeungpo-gu,2013,"POLYGON ((126.89185 37.54737, 126.90281 37.541...",영등포구,NaN
7,11180,금천구,Geumcheon-gu,2013,"POLYGON ((126.90156 37.47754, 126.90277 37.476...",금천구,NaN
8,11170,구로구,Guro-gu,2013,"POLYGON ((126.82688 37.50549, 126.83122 37.505...",구로구,NaN
9,11160,강서구,Gangseo-gu,2013,"POLYGON ((126.85984 37.57185, 126.8595 37.5682...",강서구,47.8


In [49]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

map_df.plot(
    column='폐업_3년이내',
    cmap='Reds',
    linewidth=0.8,
    ax=ax,
    edgecolor='gray',
    legend=True,
    legend_kwds={
        'label': '3년 이내 폐업률 (%)',
        'shrink': 0.6
    }
)

ax.set_title(
    '서울 구별 전체 일반음식점 기준\n3년 이내 폐업률',
    fontsize=16
)

ax.axis('off')
plt.show()

AttributeError: 'RcParams' object has no attribute '_get'